In [ ]:
import pandas as pd

# import stravalib
import flask
from flask import request
from stravalib import Client
import requests
import json
import webbrowser
import pandas as pd
import datetime

from bokeh.plotting import figure, output_file, show, output_notebook, ColumnDataSource
from bokeh.palettes import Spectral11
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.layouts import gridplot
from bokeh.core.properties import value
# from ... import strava_creds

import warnings
warnings.simplefilter('ignore')

In [ ]:
with open('../strava_creds/strava_creds.json') as scr:
    strava_creds = json.load(scr)

In [ ]:
strava_creds

In [ ]:
client = Client()
# authorize_url = client.authorization_url(client_id=24706, redirect_uri='http://localhost:8282/authorized')

# url = client.authorization_url(client_id=24706,redirect_uri='http://127.0.0.1:5000/authorization')
client_id = strava_creds['CLIENT_ID']

access_token = strava_creds['ACCESS_TOKEN']
port = 5000
url = 'http://localhost:%d/authorized' % port
authorize_url = client.authorization_url(client_id=client_id, redirect_uri=url)

webbrowser.open(authorize_url)

code = strava_creds['CODE_MAC']

access_token = client.exchange_code_for_token(client_id=client_id, client_secret=access_token, code=code)


In [ ]:
# activities_returned = []
# activity_returned = dict()

In [ ]:
activities_list = [(activity.start_date,activity.name,activity.distance) for activity in client.get_activities(after = "2018-01-01T00:00:00Z",  limit=800)]


In [ ]:
extract_df = pd.DataFrame.from_dict(activities_list)
len(extract_df)

In [ ]:
extract_df = extract_df.rename(columns={0: 'run_date',
                          1: 'run_title',
                          2: 'run_distance_m'})

In [ ]:
extract_df['kms'] = extract_df['run_distance_m']/1000

In [ ]:
extract_df.index = extract_df['run_date']

In [ ]:
extract_df = extract_df[['run_title', 'run_distance_m', 'kms']]

In [ ]:
daily_runs_series = extract_df.resample('D')['kms'].sum()

In [ ]:
daily_runs_df = pd.DataFrame(daily_runs_series)

In [ ]:
def to_float(number):
    return float(number)

In [ ]:
def convert_week_number(week_number):
    return f'week {week_number}'

In [ ]:
def cumulative_per_week(df, week_number):
    df = df.copy(deep=True)
    df_week = df[df['week_number']==week_number]
    
    df_week['weekly_actual_total'] = df_week['kms'].sum()
    df_week['weekly_actual_cumulative'] = df_week['kms'].cumsum()
    df_week['percentage_to_total_weekly'] = df_week['weekly_actual_cumulative']/df_week['weekly_actual_total']
    df_week['weekly_goal_total'] = df_week['daily_goal'].sum()
    df_week['weekly_goal_cumulative'] = df_week['daily_goal'].cumsum()
    df_week['percentage_to_total_weekly_goal'] = df_week['weekly_goal_cumulative']/df_week['weekly_goal_total']
#     df_week['weekly_goal_left'] = df_week['weekly_goal_total'] - df_week['weekly_actual_cumulative']
    
    day_number_range = range(1, len(df_week)+1)
    if len(df_week) == 7:
        day_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        df_week['day_of_week'] = day_of_week
    else:
        df_week['day_of_week'] = None
        
    df_week['day_number'] = day_number_range
    
    return df_week

In [ ]:
daily_runs_df['kms'] = daily_runs_df.apply(lambda row: to_float(row['kms']), axis=1)

In [ ]:
daily_runs_df['cumulative_daily_actual_kms'] = daily_runs_df['kms'].cumsum()
daily_runs_df['daily_goal'] = 20/7
daily_runs_df['cumulative_daily_goal'] = daily_runs_df['daily_goal'].cumsum()
# daily_runs_df['daily_km_difference'] = 

In [ ]:
daily_runs_df['daily_kms_difference'] = daily_runs_df['kms']/daily_runs_df['daily_goal']
daily_runs_df['daily_kms_cumulative_difference'] = daily_runs_df['cumulative_daily_actual_kms']/daily_runs_df['cumulative_daily_goal']

In [ ]:
daily_runs_df.to_csv('strava_kms_runs_2018.csv')

In [ ]:
weekly_kms_df = daily_runs_df.resample('W-MON', closed='Right', label='Right').sum()

In [ ]:
weekly_kms_df = weekly_kms_df[['kms', 'daily_goal']]

In [ ]:
weekly_kms_df = weekly_kms_df.rename(columns={'daily_goal': 'weekly_goal'})

In [ ]:
weekly_kms_df['cumulative_weekly_kms'] = weekly_kms_df['kms'].cumsum()
weekly_kms_df['cumulative_weekly_goal_kms'] = weekly_kms_df['weekly_goal'].cumsum()

In [ ]:
weekly_kms_df['week_number'] = weekly_kms_df.index.week
daily_runs_df['week_number'] = daily_runs_df.index.week

In [ ]:
weekly_kms_df['week'] = weekly_kms_df.apply(lambda row: convert_week_number(row['week_number']), axis=1)
daily_runs_df['week'] = daily_runs_df.apply(lambda row: convert_week_number(row['week_number']), axis=1)

In [ ]:
weekly_kms_df = weekly_kms_df.round(2)
daily_runs_df = daily_runs_df.round(2)

In [ ]:
weekly_num = list(set(list(daily_runs_df['week_number'])))

In [ ]:
daily_weekly_kms_df = pd.DataFrame()

In [ ]:
for week in weekly_num:
    transition_df = cumulative_per_week(daily_runs_df, week)
    daily_weekly_kms_df = daily_weekly_kms_df.append(transition_df)
    

In [ ]:
daily_weekly_kms_df['daily_kms_cumulative_difference'] = daily_weekly_kms_df['cumulative_daily_actual_kms'] - daily_weekly_kms_df['weekly_goal_total']

In [ ]:
daily_weekly_kms_df

In [ ]:
daily_weekly_kms_df.to_csv('strava_data.csv')

In [ ]:
output_file('grid_plot.html')

In [ ]:
# output_file('strava_chart3.html')

source = ColumnDataSource(data=weekly_kms_df)

km_fig = figure(
                  plot_height=300, plot_width=600,
                  title='cumulative actual distance vs goal actual distance',
                  x_axis_label='week', y_axis_label='Kms',
                  toolbar_location=None)


km_fig.line(x='week_number', y='cumulative_weekly_kms', 
         color='blue', line_width=1,
         legend='Cumulative weekly kms',source=source)

km_fig.vbar(x='week_number', bottom=0, top='cumulative_weekly_kms', 
         color='blue', width=0.75, 
         legend='Actual', source=source)


km_fig.line(x='week_number', y='cumulative_weekly_goal_kms', 
         color='red', line_width=1,
         legend='Cumulative goal kms',source=source)

km_fig.legend.location = 'top_left'

# show(km_fig)

In [ ]:
x_axis = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [ ]:
day_week_cds = ColumnDataSource(daily_weekly_kms_df)


km_week_comp_fig = figure(plot_height=300, plot_width=600,
                  title='cumulative actual distance vs goal actual distance',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 50))

week_1_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])
week_2_view = CDSView(source=day_week_cds,
                        filters=[GroupFilter(column_name='week', group='week 1.0')])

km_week_comp_fig.line(x='day_of_week', y='cumulative_daily_actual_kms', 
         color='blue', line_width=1,
         legend='Cumulative',  source=day_week_cds, view=week_1_view)

km_week_comp_fig.line(x='day_of_week', y='cumulative_daily_goal', 
         color='red', line_width=1,
         legend='Cumulative goal', source=day_week_cds, view=week_2_view)

# Move the legend to the upper left corner
km_week_comp_fig.legend.location = 'top_left'

# Show the plot
# show(km_week_comp_fig)

In [ ]:
weekly_kms_df['number'] = range(1, len(weekly_kms_df)+1)

In [ ]:
# output_file('strava_chart3.html')

cds_bar = ColumnDataSource(data=weekly_kms_df)

# Create a figure with a datetime type x-axis
fig = figure(title='actual versus goals',
             plot_height=300, plot_width=600,
             x_axis_label='Week Number', y_axis_label='KMs',
             x_minor_ticks=2, y_range=(0, 50),
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
fig.vbar(x='number', bottom=0, top='kms', 
         color='blue', width=0.75, 
         legend='Actual', source=cds_bar)

# The cumulative sum will be a trend line
fig.line(x='number', y='weekly_goal', 
         color='gray', line_width=1,
         legend='Goal', source=cds_bar)

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

tooltips = [
            ('Kilometers','@kms'),
            ('Week number', '@week'),
           ]

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips))


### Weekly bar line to show atual vs goal kms


In [ ]:
# output_file('strava_chart_grid.html')

cds_bar = ColumnDataSource(data=daily_weekly_kms_df)

week_1_view = CDSView(source=cds_bar,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])

# Create a figure with a datetime type x-axis
km_week_fig = figure(title='actual versus goals',
             plot_height=300, plot_width=600,
             x_axis_label='Day of the week 1.0', y_axis_label='KMs',
             x_range=x_axis,
             x_minor_ticks=2, y_range=(0, 50),
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
km_week_fig.vbar(x='day_of_week', bottom=0, top='weekly_actual_cumulative', 
         color='blue', width=0.75, 
         legend='Actual', source=cds_bar, view=week_1_view)

# The cumulative sum will be a trend line
km_week_fig.line(x='day_of_week', y='weekly_goal_cumulative', 
         color='gray', line_width=1,
         legend='Goal', source=cds_bar, view=week_1_view)

# Put the legend in the upper left corner
km_week_fig.legend.location = 'top_left'

tooltips = [
            ('Cumulative Kilometers','@weekly_actual_cumulative'),
            ('Week number', '@week'),
           ]
# Add the HoverTool to the figure
km_week_fig.add_tools(HoverTool(tooltips=tooltips))

# Let's check it out

# all_week_gridplot = gridplot([[fig, km_fig], [km_week_comp_fig, km_week_fig]], 
#                               toolbar_location='right')
# # show(km_week_fig)
# # show(km_fig)
# # show(fig)
# show(all_week_gridplot)

In [ ]:
day_week_cds = ColumnDataSource(daily_weekly_kms_df)


weeks_comp_fig = figure(plot_height=300, plot_width=600,
                  title='Week 1 vs Week 2',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 20))

week_1_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])
week_2_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 2.0')])

weeks_comp_fig.line(x='day_of_week', y='kms', 
         color='blue', line_width=1,
         legend='Week 1',  source=day_week_cds, view=week_1_view)

weeks_comp_fig.line(x='day_of_week', y='kms', 
         color='red', line_width=1,
         legend='Week 2', source=day_week_cds, view=week_2_view)

# Move the legend to the upper left corner
weeks_comp_fig.legend.location = 'top_left'

tooltips = [
            (' Kilometers','@kms'),
           ]
# Add the HoverTool to the figure
weeks_comp_fig.add_tools(HoverTool(tooltips=tooltips))



In [ ]:
stacked_bar = figure

In [ ]:
all_week_gridplot = gridplot([[fig, km_fig], [weeks_comp_fig, km_week_fig], [None, km_week_comp_fig]], 
                              toolbar_location='right')
# show(km_week_fig)
# show(km_fig)
# show(fig)
show(all_week_gridplot)

In [ ]:
output_file('select_week.html')

In [ ]:
daily_weekly_kms_df['daily_kms_cumulative_difference_abs'] = daily_weekly_kms_df['daily_kms_cumulative_difference'].abs()

In [ ]:
df4 = daily_weekly_kms_df[daily_weekly_kms_df['week']=='week 2.0']


In [ ]:
df4 = df4[['daily_kms_cumulative_difference_abs',
              'weekly_actual_cumulative', 'day_of_week']]

In [ ]:
output_file("stacked.html")

source = ColumnDataSource(df4)

# week_2_view = CDSView(source=source,
#                         filters=[GroupFilter(column_name='week', group='week 2.0')])

p = figure(x_range=x_axis, plot_height=250, title="runs by day",
           toolbar_location=None, tools="")

colors = ['red', 'blue']
km_type = ['daily_kms_cumulative_difference_abs', 'weekly_actual_cumulative']

p.vbar_stack(km_type, x='day_of_week', width=0.9, color=colors, 
              source=source)
#               view=week_2_view)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"


In [ ]:
show(p)

In [ ]:
categories = ["Cat 1", "Cat 2"]
sub_categories = ["Sub 1", "Sub 2"]

data = {'Categories' : categories,
        'Sub 1'   : [1, 3],
        'Sub 2'   : [2, 2]}

In [ ]:
datadf = pd.DataFrame(data)

In [ ]:
datadf

In [ ]:
 p.hbar_stack(sub_categories, y='Categories', height=0.9, color=['red', 'green'], source=ColumnDataSource(data))